In [ ]:
epochs, batch, lr = 100, 32, 1e-4
num_clients, in_channels, num_classes = 6, 1, 10
Dataset, Model, Prefix = 'MNIST', 'CNN_3', 'Random'
Gdrive = '/content/drive/MyDrive/Colab Notebooks/SplitLearning3'
import torch; torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os; os.chdir(Gdrive)
%run library/Dataset.ipynb
%run library/Network.ipynb
%run library/Training.ipynb
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
Dataset = 'CIFAR'
dataset, testset = loadDataset(Dataset)
clients = np.array([1, 3, 9, 19, 30, 38])
clients = ratio_split(dataset, clients / clients.sum())

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/cifar-10-python.tar.gz to /content/
Files already downloaded and verified


In [ ]:
class SplitNN:
  def __init__(self, clients, server):
    self.clients = [c.cuda() for c in clients]
    self.server = server.cuda()

  def initialize(self, datasets, tester, epochs, batch, lr):
    for client, data in zip(self.clients, datasets):
      client.initialize(data, epochs, batch, lr)
    self.server.initialize(tester, epochs, lr)
    self.cached = [[] for c in self.clients]

  def evaluate(self):
    with torch.no_grad():
      self.server.eval()
      for client in self.clients:
        client.eval()
        client.accuracy.append(0)
        for images, labels in self.server.loader:
          output = self.server(client(images)).argmax(1)
          client.accuracy[-1] += (output == labels).sum().item()
        client.accuracy[-1] /= len(self.server.loader.dataset)

  def train_network(self, epoch, private=True, sequence=False, federate=False, caches=None, shuffle=False):
    local = lambda m, p: m.client[0] if p else m
    self.server.train()
    if shuffle:
      index = torch.randperm(len(self.clients))
    else: index = torch.arange(len(self.clients))
    clients = [self.clients[i] for i in index]
    if sequence: #SL
      model = local(clients[-1], private).state_dict()
    for idx, client in enumerate(clients):
      if caches: #Our
        index2 = index.tolist(); index2.remove(idx)
        cached = [self.cached[i] for i in index]
        maxlen = [caches[i] for i in index]
      if sequence: #SL
        local(client, private).load_state_dict(model)
      client.train()
      for images, labels in client.loader:
        client.zero_grad(), self.server.zero_grad()
        #output = self.server(client(images))
        output = client(images)
        #always cache data
        if len(labels) == batch:
          self.cached[idx] = (output.clone(), labels)
        else:
          self.cached[idx] = (torch.cat([self.cached[idx][0], output.clone()]),
                              torch.cat([self.cached[idx][1], labels]))
        if caches and epoch > 0:
          index3 = [torch.randperm(len(c)) for _,c in cached]
          images_cached = [c[0][i][:m] for c,i,m in zip(cached,index3,maxlen)]
          labels_cached = [c[1][i][:m] for c,i,m in zip(cached,index3,maxlen)]
          output = torch.cat([output] + images_cached)
          labels = torch.cat([labels] + labels_cached)
        output = self.server(output)
        loss = F.cross_entropy(output, labels)
        grads = self.server.backward(loss)
        client.backward(grads[:len(client.output)])
        #client.backward(self.server.backward(loss))
        client.optim_step(), self.server.optim_step()
      client.sched_step()
      if sequence: #SL
        model = local(client, private).state_dict()
    self.server.sched_step()
    if federate: #SFL
      models = [local(c, private).state_dict() for c in clients]
      for key in models[0]:
        for i in range(1, len(models)):
          models[0][key] += models[i][key]
        if models[0][key].type().split('.')[-1] == 'LongTensor':
          torch.div(models[0][key], len(models), rounding_mode='floor')
        else: models[0][key] /= len(models)
      for client in clients:
        local(client, private).load_state_dict(models[0])

In [ ]:
def train_splitnn(clients, testset, method, epochs=epochs, model=Model, batch=batch, lr=lr, caches=None, shuffle=False):
  in_channels, num_classes, tester = loadTester(testset)
  models = [globals()[model](in_channels, num_classes) for c in clients]
  model = SplitNN([Client(m.client) for m in models], Server(models[0].server))
  model.initialize(clients, tester, epochs, batch, lr)

  for epoch in tqdm(range(epochs)):
    if method == 'private': model.train_network(epoch, private=True, sequence=False, federate=False, caches=caches, shuffle=shuffle)
    if method == 'sequence': model.train_network(epoch, private=False, sequence=True, federate=False, caches=caches, shuffle=shuffle)
    if method == 'sequence_private': model.train_network(epoch, private=True, sequence=True, federate=False, caches=caches, shuffle=shuffle)
    if method == 'federate': model.train_network(epoch, private=False, sequence=False, federate=True, caches=caches, shuffle=shuffle)
    if method == 'federate_private': model.train_network(epoch, private=True, sequence=False, federate=True, caches=caches, shuffle=shuffle)
    model.evaluate()
    plot_progress(model.clients, locals())
  return model

In [ ]:
Model = 'VGG_11'
saveAccuracies(Gdrive, train_splitnn(clients, (testset,3,10), 'private', model=Model, shuffle=True).clients, 'ImB-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Random-6-ImB-Prv-Rnd.acc


In [ ]:
caches = [8 for i in range(num_clients)]
saveAccuracies(Gdrive, train_splitnn(clients, (testset,3,10), 'private', model=Model, caches=caches, shuffle=True).clients, 'ImB-Prv-Cac')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Random-6-ImB-Prv-Rnd-Cac.acc


In [ ]:
clients = torch.ones((num_clients,))
clients = ratio_split(dataset, clients / clients.sum())
saveAccuracies(Gdrive, train_splitnn(clients, (testset,3,10), 'private', model=Model, shuffle=True).clients, 'IID-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Random-6-IID-Prv.acc
